<a href="https://colab.research.google.com/github/skybristol/experiments/blob/dev/Packaging_URLs_Files_for_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

One of the design patterns for a standard service we are exploring is where we need to fetch data from some online location where a file is available via a URL, put those files into a cloud bucket, and then kick off various types of transformation processing. In one use case, we are pulling files linked to from an inventory page for a USGS time-series data set put out as a periodic "Data Series Report." We want to get these data into a combined format with an API for working with the data more effectively than regularly stitching together individual files.

This notebook looks at how to read in the web page where the files are linked and prepare messages for a message queue process that will kick off file retrieval over HTTP onto S3 and then launch subsequent processing. In this and many cases, we will need to assemble some context from the way that the files are distributed into metadata about the files that can be used to drive specific processing. Often in these cases, we will be dealing with file contents that are not individually described in a way that is sufficient to understand and work with them effectively. These are essentially cases where data are treated as "ancillary files" to a publication or web page as opposed to being thoroughly documented and served through a data repository. We're trying to get away from this practice, but it is still all too common and we need some way of dealing with that into further processing.

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
ds140_url = "https://www.usgs.gov/centers/nmic/historical-statistics-mineral-and-material-commodities-united-states"
r = requests.get(ds140_url)
page_content = BeautifulSoup(r.text, 'html.parser')

This is where we essentially need to apply some customized logic for every case like this to create a metadata structure with whatever information we can derive from the context and/or the source content. We will want to hone this and standardize it as much as possible, potentially working to align the structure with some appropriate metadata standard. In the near term, I'm working on just coming up with the information that could be used to generate a set of messages for a message queue that would kick off processing. We need to develop enough context around each file we identify behind URLs on the subject source page that subsequent processes know how to operate and what information to pass along.

In [40]:
inventory = list()

for row in page_content.find_all("table")[1].find_all("tr")[2:]:
    columns = row.find_all("td")
    commodity_name = columns[0].text.strip().replace("\xa0", " ")
    if columns[2].text.strip().lower() in ["xlsx","xls"]:
        inventory.append({
            "commodity": commodity_name,
            "data_source": "DS140",
            "data_source_url": "https://www.usgs.gov/centers/nmic/historical-statistics-mineral-and-material-commodities-united-states",
            "file_source_extraction_function": "<resolvable reference to function that generated this message>",
            "data_type": "Supply-Demand Statistics",
            "update_year": columns[3].text.strip(),
            "file_url": columns[2].find("a")["href"],
            "next_actions": [
                "Extract Worksheets as CSV Tables",
                "Extract Word Documents"
            ]
        })

    if len(columns) >= 6 and columns[5].text.strip().lower() in ["xlsx","xls"]:
        inventory.append({
            "commodity": commodity_name,
            "data_source": "DS140",
            "data_source_url": "https://www.usgs.gov/centers/nmic/historical-statistics-mineral-and-material-commodities-united-states",
            "file_source_extraction_function": "<resolvable reference to function that generated this message>",
            "data_type": "End-Use Statistics",
            "update_year": columns[-1].text.strip(),
            "file_url": columns[5].find("a")["href"],
            "next_actions": [
                "Extract Worksheets as CSV Tables"
            ]
        })

    if len(columns) == 5 and columns[3].text.strip().lower() in ["xlsx","xls"]:
        inventory.append({
            "commodity": commodity_name,
            "data_source": "DS140",
            "data_source_url": "https://www.usgs.gov/centers/nmic/historical-statistics-mineral-and-material-commodities-united-states",
            "file_source_extraction_function": "<resolvable reference to function that generated this message>",
            "data_type": "End-Use Statistics",
            "update_year": columns[-1].text.strip(),
            "file_url": columns[3].find("a")["href"],
            "next_actions": [
                "Extract Worksheets as CSV Tables"
            ]
        })


The resulting dictionaries here should be relatively simple and straightforward. Here is a basic explanation of each property we've assembled and why it is important to the next steps.

* commodity - commodity names (minerals and other materials) are the key organizing identifier that will be used in our system. These will ultimately need to be aligned with a source of definition and a resolvable identifier from larger vocabulary work, but we do a little cleanup here on how they are listed in this venue.
* data_source - simple referral point to how this data source is often used in common language
* data_source_url - it's important to keep track of what was processed to get the references we are sending on for further processing
* file_source_extraction_function - this is a placeholder for some type of resolvable reference that points to the Lambda code or some executable that did the work against the data_source_url to get files for processing
* file_url - the URL where a file that we want to fetch into further processing can be obtained
* next_actions - text strings here should probably be replaced with some kind of resolvable identifier for subsequent processes (executed as lambdas or whatever) that are going to be kicked off after the files are fetched onto cloud storage; in this case we are saying that all files need to have CSV tables extracted and some also need to be checked for embedded Word documents and have those pulled out
* update_year - this is the key piece of information that indicates when a given commodity statistics file was updated, we may need to count on this as an indicator of when a file should be re-fetched for re-processing as the file URL and other information may not be sufficient

Note on file naming: It might be reasonable to use a hash of the file_url value as a file name/prefix here as opposed to what will be variable source file names. This would help us connect dots with an opaque unique identifier back to metadata from this process. In running subsequent extraction processes to do things like pull out CSV tables where we might have multiple CSV files per Excel file, we could use a combination of the hashed file_url with worksheet name or index position to uniquely name those derived files.

In [41]:
inventory

[{'commodity': 'Abrasives (manufactured)',
  'data_source': 'DS140',
  'data_source_url': 'https://www.usgs.gov/centers/nmic/historical-statistics-mineral-and-material-commodities-united-states',
  'data_type': 'Supply-Demand Statistics',
  'file_source_extraction_function': '<resolvable reference to function that generated this message>',
  'file_url': 'https://prd-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/atoms/files/ds140-2017-abras.xlsx',
  'next_actions': ['Extract Worksheets as CSV Tables',
   'Extract Word Documents'],
  'update_year': '2017'},
 {'commodity': 'Abrasives (natural)',
  'data_source': 'DS140',
  'data_source_url': 'https://www.usgs.gov/centers/nmic/historical-statistics-mineral-and-material-commodities-united-states',
  'data_type': 'Supply-Demand Statistics',
  'file_source_extraction_function': '<resolvable reference to function that generated this message>',
  'file_url': 'https://s3-us-west-2.amazonaws.com/prd-wret/assets/palladium/production/